In [1]:
import requests
import random
import json

base_url = "http://localhost:8081"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [2]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200


In [3]:
resp.json()

{'SchoolCertificate': {'$id': '#/properties/SchoolCertificate',
  'type': 'object',
  'title': 'The School Certificate Schema',
  'required': ['name', 'certificateName'],
  'properties': {'name': {'type': 'string', 'title': 'Name of the Student'},
   'studentId': {'type': 'integer', 'title': 'The ID of the Student'},
   'certificateName': {'type': 'string',
    'enum': ['Common Entrance',
     'JSCE',
     'Junior NECO',
     'SSCE - WAEC',
     'SSCE - NECO',
     'SSCE - NABTEB',
     'NCE',
     'OND',
     'HND',
     'B.Sc',
     'B.Ed',
     'B.Tech',
     'MBBS',
     'B.A',
     'LLB',
     'PGDE',
     'M.Sc',
     'M.Tech',
     'MBA',
     'M.Ed',
     'M.Ph',
     'M.Phil',
     'PhD.'],
    'title': 'The name of the specific certificate to be issued to the student'},
   'academicYear': {'type': 'string', 'title': 'Academic Year'},
   'dateofaward': {'type': 'string',
    'title': 'Date Of Award',
    'format': 'date'}}}}

In [4]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['SchoolCertificate']


In [5]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity SchoolCertificate


In [8]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/%s'%entity_name), json={
  "name": "Sunbird Learner",
  "certificateName": "Junior NECO",
  "studentId": 101,
  "academicYear": "2022/2023",
  "dateofaward": "2024-09-16"
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"][entity_name]["osid"]


{'id': 'sunbird-rc.registry.post', 'ver': '1.0', 'ets': 1726474949931, 'params': {'resmsgid': '', 'msgid': '1704b3c8-5b6c-47df-95d7-31fb7865e53f', 'err': '', 'status': 'UNSUCCESSFUL', 'errmsg': 'java.lang.RuntimeException: Timeout lapsed to acquire write lock for notification.'}, 'responseCode': 'OK', 'result': {}}


AssertionError: 

In [ ]:
resp.json()


In [ ]:
print(resp.json())
resp.status_code, userId


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf"})


In [ ]:
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)
    

In [ ]:
%system open 'sample.pdf'


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

# generate certificate by external templates

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf", "template": "https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample_external.pdf', 'wb') as f:
    f.write(resp.content)
%system open 'sample_external.pdf'


# delete certificate

In [ ]:
esp = requests.delete("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(resp.status_code)
assert resp.status_code == 200
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(json.dumps(resp.json()))
assert resp.json()["params"]["errmsg"] == "entity status is inactive"


# generate credentials with external template

In [ ]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/SkillCertificate'), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module",
    "skills": [
        {
            "skill": "computer training",
            "certifiedOn": "12-12-2000"
        },
        {
            "skill": "network training",
            "certifiedOn": "12-11-2000"
        }

    ]
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"]["SkillCertificate"]["osid"]
print(resp.json())

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/pdf", "template": "http://localhost:63342/sunbird-rc/registry/public/_schemas/templates/SkillCertificate.html?_ijt=qu4536mc0j899963igug2gitf0"})

resp.status_code, resp.content
print(resp)
assert resp.content[:5].decode().startswith("%PDF")
with open('sample1.pdf', 'wb') as f:
    f.write(resp.content)
    
%system open 'sample1.pdf'
